# Laboratorio Regresión Lineal


In [3]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree

# Para realizar la regresión lineal y la evaluación del modelo
# -----------------------------------------------------------------------
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


from sklearn.model_selection import KFold,LeaveOneOut, cross_val_score


from sklearn.preprocessing import StandardScaler

from tqdm import tqdm


# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = "{:,.2f}".format

import sys
sys.path.append("../")
from src.soporte_metricas import (
    obtener_metricas
)

# Vamos a realizar Cross Validation
Usando el encoding Target
- El objetivo es ver si nuestro modelo puede mejorar al realizar varias iteraciones. de la regresión Lineal.

In [4]:
df = pd.read_csv("../datos/06_autorenew_target_encoded.csv",index_col=0)
df.sample()

,offerType,vehicleType,fuelType,gearbox,notRepairedDamage,kilometer,price,yearOfRegistration,gama,powerCV_robust_scaler
243820,"6,145.40","5,539.81","5,153.38","4,956.84","3,756.20","15,557.50","10,950.00","12,436.82","7,471.21","9,912.23"


In [5]:
X = df.drop(columns="price")
y = df[["price"]]
X_train, X_test, y_train, y_test = train_test_split(X, # Todo menos la Variable Respuesta
                                                    y, # La variable Respuesta
                                                    train_size=0.7, # Como dividir los datos en este caso 80% train, 20% test
                                                    random_state=42, # Semilla, para que lo haga igual para cualquier persona
                                                    shuffle=True 
                                                    )


# Hagamos el modelo Linear y Recapitulemos

In [6]:
modelo_lineal = LinearRegression()
modelo_lineal.fit(X_train, y_train)
y_pred_test = modelo_lineal.predict(X_test)
y_pred_train = modelo_lineal.predict(X_train)

# Veamos sus métricas

In [7]:
obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)

,r2_score,MAE,MSE,RMSE
train,0.66,"2,762.76","19,231,671.73","4,385.39"
test,0.65,"2,738.64","18,515,822.91","4,303.00"


### Como vimos anteriormente:
- No es viable el error que tenemos ni el R2 es aceptable
- Para solucionarlo, vamos a intentar realizar K-Fold Cross Validation
### ¿Porque?
- Una de las principales causas es que tenemos un conjunto de más de 100.000 datos, lo que lo hace un conjunto moderado de datos, por rendimiento y tiempo empezaremos por este

In [8]:
df.shape

(334843, 10)

### ¿Que puedo meter en scoring?

- r2
- MAE: neg_mean_absolute_error
- MSE: neg_mean_squared_error
- RMSE: neg_root_mean_squared_error



# Empecemos calculando el R2
- En varias iteraciones veremos si mejora potencialmente
### ¿Cuántos splits?
- Voy a probar con 5, 50, 100, 500 y 1000, de ahí vemos la media de r2


In [9]:
splits = [5,50,100,500,1000]
for split in tqdm(splits):
    kf = KFold(n_splits=split,
            shuffle=True,
            random_state=42)
    score_r2 = cross_val_score(modelo_lineal,
                            X,
                            y,
                            cv=kf,
                            scoring="r2")
    print(f"r2 en {split} splits: {np.mean(score_r2)}")

 20%|██        | 1/5 [00:00<00:03,  1.24it/s]

r2 en 5 splits: 0.6547545572457727


 40%|████      | 2/5 [00:08<00:13,  4.62s/it]

r2 en 50 splits: 0.6547879023341562


 60%|██████    | 3/5 [00:22<00:18,  9.12s/it]

r2 en 100 splits: 0.6546626062653804


 80%|████████  | 4/5 [01:38<00:35, 35.55s/it]

r2 en 500 splits: 0.6544720357940187


100%|██████████| 5/5 [03:44<00:00, 44.98s/it]

r2 en 1000 splits: 0.6541562993612292


Tarda 2 mins
- r2 en 5 splits: 0.33618296668739367
- r2 en 50 splits: 0.41857590248457505
- r2 en 100 splits: 0.44196576819632055
- r2 en 500 splits: 0.5091593582512305
- r2 en 1000 splits: 0.5178094658590221

Antes de tener decision, veamos el error RMSE y su evolución

In [10]:
splits = [5,50,100,500,1000]
for split in tqdm(splits):
    kf = KFold(n_splits=split,
            shuffle=True,
            random_state=42)
    score_rmse = cross_val_score(modelo_lineal,
                            X,
                            y,
                            cv=kf,
                            scoring="neg_root_mean_squared_error")
    print(f"Error RMSE en {split} splits: {np.mean(score_rmse)}")

 20%|██        | 1/5 [00:00<00:02,  1.54it/s]

Error RMSE en 5 splits: -4360.836208916828


 40%|████      | 2/5 [00:07<00:12,  4.05s/it]

Error RMSE en 50 splits: -4358.7988044689855


 60%|██████    | 3/5 [00:19<00:15,  7.72s/it]

Error RMSE en 100 splits: -4356.599553575688


 80%|████████  | 4/5 [01:21<00:29, 29.43s/it]

Error RMSE en 500 splits: -4339.4568148486405


100%|██████████| 5/5 [03:30<00:00, 42.10s/it]

Error RMSE en 1000 splits: -4319.007423640001


Tarda 4 mins
- Error RMSE en 5 splits: -12267.559531024217
- Error RMSE en 50 splits: -10612.957198951419
- Error RMSE en 100 splits: -10273.086514930843
- Error RMSE en 500 splits: -8429.755759419888
- Error RMSE en 1000 splits: -7765.457287420199

# La Realidad
- Aún generando grupos de 1000 con los datos, no somos capaces de reducir el error de forma sustancial
- Si es verdad que el R2 crece a más grupos, y lo más probable es que siguiera creciendo
- Por eso vemos el error RMSE, para saber si nos reduce el error
- Y vemos que aún reduciendo el error, una media de 7765€ de error el el precio sigue siendo inaceptable

# ¿Entonces?
- Vamos a realizar un Decision Tree en el jupyter 07